# Average Annual Household Transportation Costs
Direct daily out-of-pocket expenses including tolls, parking, and transit fares

These costs do not include other vehicle operating and purchase costs.


In [1]:
import pandas as pd


In [2]:
# Direct out-of-pocket costs
# Note that these do not include additional costs associated with owning a vehicle, 
# such as insurance, vehicle cost, repairs, etc.

In [3]:
df = pd.read_csv(r'../../../outputs/agg/dash/person_cost.csv')
df_hh = pd.read_csv(r'..\..\..\outputs\agg\dash\hh_geog.csv')

annual_factor = 300

df['is_rgc'] = 0
df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_hh['is_rgc'] = 0
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

In [4]:
pd.options.display.float_format = '${:0,.0f}'.format

In [5]:
_df = pd.DataFrame(df.groupby('hh_county').sum()['travcost_wt']/df_hh.groupby('hh_county').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_county,
King,"$3,249"
Kitsap,"$3,591"
Pierce,"$3,753"
Snohomish,"$4,057"


In [6]:
_df = pd.DataFrame(df.groupby('is_rgc').sum()['travcost_wt']/df_hh.groupby('is_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
is_rgc,
0,"$3,720"
1,"$1,319"


In [7]:
_df = pd.DataFrame(df.groupby('hh_rgc').sum()['travcost_wt']/df_hh.groupby('hh_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rgc,
Auburn,"$2,429"
Ballard-Interbay,"$1,522"
Bellevue,"$1,201"
Bothell Canyon Park,"$3,425"
Bremerton,"$1,123"
Burien,"$2,342"
Cascade,"$3,929"
Duwamish,"$1,834"
Everett,"$1,304"


In [8]:
_df = pd.DataFrame(df.groupby('hh_rg_proposed').sum()['travcost_wt']/df_hh.groupby('hh_rg_proposed').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rg_proposed,
CitiesTowns,"$4,694"
Core,"$3,365"
HCT,"$3,713"
Metro,"$2,281"
Rural,"$5,971"
UU,"$4,449"


In [9]:
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 1].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'])*annual_factor
            results_df.loc[equity_geog+geog_type,'costs'] = _df
        
inv_results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 0].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 0].sum()['hhexpfac'])*annual_factor
            inv_results_df.loc[equity_geog+geog_type,'costs'] = _df
        
# inv_results_df.index = ['NOT in '+i for i in results_df.index] 
_df = results_df.merge(inv_results_df, left_index=True, right_index=True)
_df.columns = ['Inside Equity Geog','Outside Equity Geog']
_df[~_df.isnull()]

,Inside Equity Geog,Outside Equity Geog
hh_youth_reg,"$4,127","$2,925"
hh_elderly_reg,"$3,600","$3,450"
hh_elderly_50,"$4,655","$3,520"
hh_english_reg,"$3,323","$3,639"
hh_racial_reg,"$3,170","$3,805"
hh_racial_50,"$3,260","$3,587"
hh_poverty_reg,"$3,175","$3,772"
hh_poverty_50,"$2,719","$3,540"
hh_disability_reg,"$3,455","$3,579"
